# Project : Custom Cohere with memory

Have you may have seen, there is no memory conversation with our previous LLM. 

Chatbots are one of the central LLM use-cases. The core features of chatbots are that they can have long-running conversations and have access to information that users want to know about.

Aside from basic prompting and LLMs, memory and retrieval are the core components of a chatbot. Memory allows a chatbot to remember past interactions, and retrieval provides a chatbot with up-to-date, domain-specific information.

In this project we will focus on memory.

You will find everything needed here : https://python.langchain.com/docs/modules/memory/ 

### Complete pre-requisite

#### Packages

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain.schema import SystemMessage
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory,
                              ConversationBufferWindowMemory,
                              ConversationKGMemory)
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


# You can add other package here if needed
    

#### Initialize LLM & ConversationChain

A ConversationChain is a type of chain that is specifically designed to handle conversations or dialogues, as opposed to individual inputs.

In [3]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
COHERE_API_KEY = os.environ.get('COHERE_API_KEY')
llm = Cohere(max_tokens=512,cohere_api_key=COHERE_API_KEY)

# We will use a ConversationChain for our project
conversation = ConversationChain(llm=llm)
print(conversation.prompt.template)


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.



### ConversationBufferMemory

The conversation buffer memory keeps the previous pieces of conversation completely unmodified, in their raw form.

In [10]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Création d'une instance de ConversationBufferMemory
conversation_buffer_memory = ConversationBufferMemory()

# Initialisation d'une ConversationChain avec le modèle de langage et la mémoire
conversation_with_buffer_memory = ConversationChain(llm=llm, memory=conversation_buffer_memory)

    

#### verification

In [11]:
conversation_with_buffer_memory.invoke("What is the capital of France ?")

conversation_with_buffer_memory.invoke("What is the capital of Spain ?")

conversation_with_buffer_memory.invoke("What is the capital of Angola ?")

# We should be able to get the whole conversation inside the memory
print(conversation_with_buffer_memory.memory.buffer)

Human: What is the capital of France ?
AI:  The capital of France is Paris. It is located in the north-central part of the country, on the River Seine, and is one of the most populous and well-known cities in the world.

Paris is divided into 20 arrondissements (administrative districts) surrounded by the broader Parisian agglomeration. Historical landmarks include the Eiffel Tower, Notre-Dame Cathedral, the Louvre Museum, the Arc de Triomphe, and the Palace of Versailles. 

It is known for its fashion, art, cuisine, and culture. Would you like to know more about Paris ? 
Human: What is the capital of Spain ?
AI:  The capital of Spain is Madrid. It is located in the central part of the country, on the Manzanares River, and is also one of the most populous and famous cities in the world.

Madrid serves as an important political, economic, cultural, and educational center not only for Spain but also for Southern Europe and the Latin American region. It is host to the Spanish Royal Family

#### From your observations - Gives the pros and cons of this memory 

##### Pros

- Stockage de l'historique de la conversation : La mémoire conserve un enregistrement des questions posées par l'utilisateur et des réponses fournies par l'IA. Cela permet à l'IA de se souvenir du contexte de la conversation et d'adapter ses réponses en conséquence.
- En conservant une trace des questions précédentes, l'IA peut apprendre et s'améliorer avec le temps. Si elle ne peut pas répondre à une question lors d'une interaction, elle peut utiliser les informations stockées dans la mémoire pour enrichir sa base de connaissances

##### Cons

- Si la taille du buffer est trop petite, l'IA risque de perdre le contexte des interactions passées plus rapidement, ce qui pourrait entraîner des réponses moins pertinentes ou cohérentes dans le cadre de la conversation.
- La qualité des réponses de l'IA dépend fortement de la qualité des interactions précédentes stockées dans la mémoire. Si les questions posées précédemment étaient mal formulées ou si les réponses étaient incorrectes, cela pourrait affecter négativement les performances de l'IA dans les interactions futures



### ConversationSummaryMemory

The conversation summary memory creates a summary of the conversation over time. 

In [14]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

conversation_with_summary_memory = ConversationSummaryMemory(llm=llm)
converstation_summary_memory = ConversationChain(llm=llm, memory=conversation_with_summary_memory)
    

#### Verification

In [15]:
converstation_summary_memory.invoke("Gives me the historical chronology of France. I want as much dates and events as possibles.")

converstation_summary_memory.invoke("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

converstation_summary_memory.invoke("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

# We should be able to get the whole conversation inside the memory
print(converstation_summary_memory.memory.buffer)

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.
Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.


 The human asks the AI to provide a detailed historical chronology of Germany with dates and events. The AI lists six eras with noteworthy dates and events, from prehistory to the Holy Roman Empire. Some important social, political, and economic milestones include the fall of the Roman Empire, the Black Death, and the rise of Christianity in Spain. 


#### From your observations - Gives the pros and cons of this memory 

##### Pros

- Résumé concit qui facilite l'obtention des informations rapidement
- Mise en évidence des élèments clés et pértinents de la conversation 

##### Cons

- Perte de détails par rapport à une mémoire qui stocke l'historique complet de chaque interaction
- Limitation de la rétroaction et de l'apprentissage : En résumant les interactions, la mémoire ConversationSummaryMemory peut limiter la quantité de rétroaction et d'apprentissage que l'IA peut tirer des interactions passées



### ConversationSummaryBufferMemory

The conversation summary buffer memory keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions, it compiles them into a summary and uses both.

In [17]:

from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

conversation_summary_buffer_memory = ConversationSummaryBufferMemory(llm=llm)
conversation_with_summary_buffer_memory = ConversationChain(llm=llm, memory=conversation_summary_buffer_memory)

    

#### Verification

In [18]:
conversation_with_summary_buffer_memory.invoke("Gives me the historical chronology of France. I want as much dates and events as possibles.")

conversation_with_summary_buffer_memory.invoke("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

conversation_with_summary_buffer_memory.invoke("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

# We should be able to get the whole conversation inside the memory
print(conversation_with_summary_buffer_memory.memory.buffer)

C:\Users\Barkaoui\Desktop\M2\Workshop-AI\myvenv3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[HumanMessage(content='Gives me the historical chronology of France. I want as much dates and events as possibles.'), AIMessage(content=' Sure! Here is an extensive list of important dates and events in French history. \n\nPrehistory - the Paleolithic and Neanderthal eras (50,000 to 10,000 BC). During the Upper Paleolithic, the Aurignacian culture brings Neanderthals to extinction through advanced stone tools, jewelry, and figurative art such as the Venus of Brassempouy.\n\n Protohistory - the Celtic and Gallic periods (1200 BC to 51 BC). The Celtic Hallstatt culture emerges around 750 BC, followed by the La Tène culture from 450 BC. Mass migrations across Europeoccur around 300 BC, including the first Celtic invasions of Italy, Greece, and Asia Minor. \n\nThe Roman period (51 BC to 486 AD). Gaul is conquered by Julius Caesar between 58 and 51 BC and becomes a Roman province. The Gallo-Roman culture emerges, followed by Christianity from around AD 200. The earliest Romanesque architect

#### From your observations - Gives the pros and cons of this memory 

##### Pros

-
-

##### Cons

-
-

